<a href="https://colab.research.google.com/github/BitKnitting/FitHome_Analysis/blob/master/notebooks/Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calculating Baseline
The active part of the FitHome experience begins with knowledge of initial consumption.

#Data from Firebase
Retrieve the readings into a json file.

In [1]:
%%time
!curl 'https://fithome-9ebbd.firebaseio.com/flower-09282019/readings.json?print=pretty' > baseline.json 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.3M  100 17.3M    0     0  2641k      0  0:00:06  0:00:06 --:--:-- 5542k
CPU times: user 43 ms, sys: 11.1 ms, total: 54.2 ms
Wall time: 7.64 s


In [4]:
# @title Put into a Pandas dataframe
# This can take >= 3 minutes.
%%time
import pandas as pd
df_needs_reshape = pd.read_json("baseline.json","r", encoding="utf8")
df_reshaped = df_needs_reshape.T
df_reshaped.head()

CPU times: user 2min 46s, sys: 4.09 s, total: 2min 50s
Wall time: 2min 43s


In [0]:
# The datetime is in UTC time.  We are in Kirkland, WA
df_reshaped.index = df_reshaped.index.tz_localize('UTC').tz_convert('US/Pacific')

In [6]:
# @title Start Date and End Data of Series
print('Start date: {}'.format(df_reshaped.index.min()))
print('End   date: {}'.format(df_reshaped.index.max()))
time_between = df_reshaped.index.max() - df_reshaped.index.min()
print('\n\nElapsed time: {}'.format(time_between))

Start date: 2019-10-11 13:46:35-07:00
End   date: 2019-10-30 15:53:37-07:00


Elapsed time: 19 days 02:07:02


# Describe the Data
Let's look at some interesting stats

In [7]:
# @title Interesting Data Stats
df_reshaped.describe()

,I,P
count,302528.000000,302528.000000
mean,6.209079,663.429233
std,4.886584,547.643994
min,0.002000,0.000320
25%,2.526000,259.655300
50%,4.340000,481.669100
75%,8.823000,913.125750
max,52.895000,5663.446000


# Daily Average Electricity Use

- goup data into days.
- use only days which have enough samples in each hour.

In [11]:
all_power_data = df_reshaped['P']
# Convert readings to kWh
p_kWh = all_power_data.resample(rule='D').mean()/1000
display(p_kWh[:5])

2019-10-11 00:00:00-07:00    0.484437
2019-10-12 00:00:00-07:00    1.081336
2019-10-13 00:00:00-07:00    0.899607
2019-10-14 00:00:00-07:00    0.558521
2019-10-15 00:00:00-07:00    0.692476
Freq: D, Name: P, dtype: float64